In [ ]:
import tkinter as tk
from tkinter import ttk
import numpy as np

# Sample user-item matrix (ratings)
user_item_matrix = np.array([
    [5, 4, 0, 0, 3],  # User 1
    [0, 5, 4, 0, 4],  # User 2
    [4, 0, 5, 4, 0],  # User 3
    [0, 4, 0, 5, 0],  # User 4
    [5, 0, 4, 0, 4]   # User 5
])

# Function to compute similarity between two users
def cosine_similarity(user1, user2):
    nonzero_indices_user1 = np.nonzero(user1)
    nonzero_indices_user2 = np.nonzero(user2)
    common_items = np.intersect1d(nonzero_indices_user1, nonzero_indices_user2)
    if len(common_items) == 0:
        return 0  
    else:
        dot_product = np.sum(user1[common_items] * user2[common_items])
        magnitude_user1 = np.linalg.norm(user1[common_items])
        magnitude_user2 = np.linalg.norm(user2[common_items])
        similarity = dot_product / (magnitude_user1 * magnitude_user2)
        return similarity

# Function to recommend items to a user
def recommend_items(user_id, user_item_matrix, k=2):
    similarities = [cosine_similarity(user_item_matrix[user_id], user_item_matrix[i]) for i in range(len(user_item_matrix))]
    similar_users_indices = np.argsort(similarities)[::-1][1:]
    unrated_items_indices = np.where(user_item_matrix[user_id] == 0)[0]
    recommendations = []
    for item_idx in unrated_items_indices:
        similar_ratings = [user_item_matrix[user_idx][item_idx] for user_idx in similar_users_indices]
        avg_rating = np.mean(similar_ratings)
        recommendations.append((item_idx, avg_rating))
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:k]

# Function to handle recommendation button click
def recommend():
    user_id = int(user_entry.get()) - 1
    recommendations = recommend_items(user_id, user_item_matrix, k=2)
    result_label.config(text="Recommendations for User " + str(user_id + 1) + ":")
    for idx, rating in recommendations:
        result_label.config(text=result_label.cget("text") + "\nItem " + str(idx + 1) + " with predicted rating: " + str(rating))

# Create GUI
root = tk.Tk()
root.title("Simple Recommendation System")

# User input
user_frame = ttk.Frame(root)
user_frame.pack(padx=10, pady=10)
user_label = ttk.Label(user_frame, text="Enter User ID:")
user_label.grid(row=0, column=0)
user_entry = ttk.Entry(user_frame)
user_entry.grid(row=0, column=1)

# Recommend button
recommend_button = ttk.Button(root, text="Recommend", command=recommend)
recommend_button.pack(pady=5)

# Result label
result_label = ttk.Label(root, text="")
result_label.pack(pady=5)

root.mainloop()
